<a href="https://colab.research.google.com/github/SaiVaichal/key_exchange/blob/main/key_exchage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from cryptography.hazmat.primitives.asymmetric import rsa, dh, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os
import base64

class KeyManagementSystem:
    def __init__(self):
        self.rsa_keys = {}
        self.dh_params = dh.generate_parameters(generator=2, key_size=2048)
        self.dh_keys = {}
        self.aes_keys = {}

    def generate_rsa_keys(self, user):
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048
        )
        public_key = private_key.public_key()
        self.rsa_keys[user] = (private_key, public_key)

    def generate_diffie_hellman_keys(self, user):
        private_key = self.dh_params.generate_private_key()
        public_key = private_key.public_key()
        self.dh_keys[user] = (private_key, public_key)

    def generate_symmetric_key(self, user):
        key = os.urandom(32)  # 256-bit AES key
        self.aes_keys[user] = key

    def diffie_hellman_key_exchange(self, user_a, user_b):
        private_key_a, public_key_a = self.dh_keys[user_a]
        private_key_b, public_key_b = self.dh_keys[user_b]
        shared_key_a = private_key_a.exchange(public_key_b)
        shared_key_b = private_key_b.exchange(public_key_a)
        return shared_key_a, shared_key_b  # Both should be identical

    def encrypt_with_aes(self, user, plaintext):
        key = self.aes_keys[user]
        iv = os.urandom(16)
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
        encryptor = cipher.encryptor()
        padded_plaintext = plaintext + b' ' * (16 - len(plaintext) % 16)
        ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
        return base64.b64encode(iv + ciphertext).decode()

    def decrypt_with_aes(self, user, ciphertext):
        key = self.aes_keys[user]
        data = base64.b64decode(ciphertext)
        iv, encrypted_text = data[:16], data[16:]
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
        decryptor = cipher.decryptor()
        plaintext = decryptor.update(encrypted_text) + decryptor.finalize()
        return plaintext.strip()

    def revoke_key(self, user, key_type):
        if key_type == "rsa":
            self.rsa_keys.pop(user, None)
        elif key_type == "dh":
            self.dh_keys.pop(user, None)
        elif key_type == "aes":
            self.aes_keys.pop(user, None)

# Example Usage:
kms = KeyManagementSystem()
kms.generate_rsa_keys("Alice")
kms.generate_rsa_keys("Bob")
kms.generate_diffie_hellman_keys("Alice")
kms.generate_diffie_hellman_keys("Bob")
kms.generate_symmetric_key("Alice")

shared_secret_a, shared_secret_b = kms.diffie_hellman_key_exchange("Alice", "Bob")
assert shared_secret_a == shared_secret_b  # They should be identical

# Encrypt and Decrypt Example
encrypted_message = kms.encrypt_with_aes("Alice", b"Hello, Secure World!")
decrypted_message = kms.decrypt_with_aes("Alice", encrypted_message)
print("Decrypted Message:", decrypted_message.decode())



Decrypted Message: Hello, Secure World!
